In [1]:
import qgrid
import pandas as pd
import numpy as py
from drg_connect import Snowflake
 
snow = Snowflake(role ='RWD_ANALYTICS_RW',database='PROJECT_ANALYTICS',schema ='AMGEN_ONCOLOGY')
engine = snow.engine
#%reload_ext sql_magic  
%reload_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

## Uploading diagnosis codes

In [129]:
Amgen_Code_List = pd.read_excel("C:/Amgen/Amgen_diag_code.xlsx",dtype=str, head=True)

In [130]:
snow.upload_dataframe(Amgen_Code_List, 'Amgen_Code_List', schema = 'AMGEN_ONCOLOGY')

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Table already exists. Aborting upload.


In [131]:
%%read_sql
create or replace table Amgen_diag_code
AS
Select 
A.*,
B.LEVEL_1_DESCRIPTION,
B.LEVEL_2_DESCRIPTION,
B.LEVEL_3_DESCRIPTION,
B.LEVEL_4_LONG_DESCRIPTION_ICD10
From Amgen_Code_List A LEFT JOIN RWD_DB.RWD_REFERENCE_LIBRARY.IG_ICD_GROUPED_DESCRIPTIONS B
ON A.code=B.code

Query started at 08:45:27 PM India Standard TimeInitiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
; Query executed in 0.25 m

status
0  Table AMGEN_DIAG_CODE successfully created.

## Diagnosis data pull from year 2011 to 2012

In [132]:
%%read_sql

create or replace table Amgen_Target_Diagnosis_All_Year as

select  A.Claim_number
       ,Patient_id
       ,A.diagnosis
       ,year_of_service as diagnosis_date
       ,B.category_1
       ,B.code_group
       ,B.Description             
from RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS A inner join Amgen_diag_code B
     on upper(A.diagnosis)=upper(B.code)
where upper(B.category_1)='ICD10'
    AND year(year_of_service) in ('2011','2012','2013','2014','2015','2016','2017','2018')
    AND Patient_id is not null

Query started at 08:45:43 PM India Standard Time; Query executed in 5.81 m

status
0  Table AMGEN_TARGET_DIAGNOSIS_ALL_YEAR successf...

### Extracting diagnosis data for one year from Amgen_Target_Diagnosis_All_Year table

In [133]:
%%read_sql
create or replace table Amgen_Target_Diagnosis
AS
Select * 
From Amgen_Target_Diagnosis_All_Year
where diagnosis_date>= '2017-11-01' AND diagnosis_date<= '2018-10-31' 

Query started at 08:51:31 PM India Standard Time; Query executed in 0.26 m

status
0  Table AMGEN_TARGET_DIAGNOSIS successfully crea...

In [134]:
%%read_sql
select count(distinct claim_number),count(distinct patient_id) from Amgen_Target_Diagnosis

Query started at 08:51:47 PM India Standard Time; Query executed in 0.12 m

COUNT(DISTINCT CLAIM_NUMBER)  COUNT(DISTINCT PATIENT_ID)
0                      29587495                     3579481

### Extracting data from Aron internal initiative table based on diagnosis codes

In [135]:
%%read_sql
create or replace table Aron_PatientTest_Table
AS
Select
A.*,
B.code_group,
B.description,
B.category_1
 from PROJECT_ANALYTICS.DRG_ONCOLOGY.cancer_claim_patient_list A inner join Amgen_diag_code B
 ON A.diagnosis=B.code
 where year_of_service>= '2017-11-01' AND year_of_service<= '2018-10-31' 

Query started at 08:51:54 PM India Standard Time; Query executed in 0.29 m

status
0  Table ARON_PATIENTTEST_TABLE successfully crea...

In [136]:
%%read_sql
select count(distinct claim_number),count(distinct patient_id) from Amgen_Target_Diagnosis 

Query started at 08:52:11 PM India Standard Time; Query executed in 0.04 m

COUNT(DISTINCT CLAIM_NUMBER)  COUNT(DISTINCT PATIENT_ID)
0                      29587495                     3579481

In [137]:
%%read_sql
select count(distinct claim_number),count(distinct patient_id) from Aron_PatientTest_Table 

Query started at 08:52:14 PM India Standard Time; Query executed in 0.14 m

COUNT(DISTINCT CLAIM_NUMBER)  COUNT(DISTINCT PATIENT_ID)
0                      28705021                     3549414

In [138]:
%%read_sql
select Count(distinct claim_number),count(distinct patient_id),month(diagnosis_date)
from Amgen_Target_Diagnosis
group by month(diagnosis_date)
order by  month(diagnosis_date) asc

Query started at 08:52:23 PM India Standard Time; Query executed in 0.06 m

COUNT(DISTINCT CLAIM_NUMBER)  COUNT(DISTINCT PATIENT_ID)  \
0                        2724948                      871343   
1                        2429951                      818055   
2                        2538392                      849725   
3                        2495831                      859520   
4                        2568508                      863528   
5                        2432058                      831616   
6                        2434531                      818764   
7                        2574919                      856725   
8                        2217355                      786305   
9                        2664686                      890226   
10                       2308106                      808117   
11                       2218073                      775937   

    MONTH(DIAGNOSIS_DATE)  
0                       1  
1                       2  
2                       3  
3                       4  
4                       5  
5                       6  
6                       7  
7                       8  
8                       9  
9                      10  
10                     11  
11                     12

### Extracting Provider data for diagnosis data

In [139]:
%%read_sql
Create or replace table Amgen_provider
AS
select A.*,
      B.provider_npi as physician_npi,
      B.provider_type
      
            
from Amgen_Target_Diagnosis A inner join RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROVIDER B
     on A.claim_number=B.claim_number  
     where 
               entity_type_code='1'
         AND   length(B.provider_npi) = 10
         and B.provider_npi not ilike '%A%'
         and B.provider_npi not ilike '%B%'
         and B.provider_npi not ilike '%L%'
         and B.provider_npi not ilike '%*%'
         and B.provider_npi not ilike '%+%'
         and B.provider_npi not ilike '% %'
         and B.provider_npi not ilike '%Y%'
         and B.provider_npi not ilike '%X%'
      

Query started at 08:52:27 PM India Standard Time; Query executed in 5.34 m

status
0  Table AMGEN_PROVIDER successfully created.

In [140]:
%%read_sql
select count(distinct claim_number),count(distinct patient_id),count(distinct physician_npi) from Amgen_provider

Query started at 08:57:47 PM India Standard Time; Query executed in 0.09 m

COUNT(DISTINCT CLAIM_NUMBER)  COUNT(DISTINCT PATIENT_ID)  \
0                      28055150                     3509410   

   COUNT(DISTINCT PHYSICIAN_NPI)  
0                         684867

In [141]:
%%read_sql

select Patient_id,description,count(distinct claim_number)
from AMGEN_PROVIDER 
group by Patient_id,description
order by count(distinct claim_number) desc

Query started at 08:57:52 PM India Standard Time; Query executed in 1.09 m

patient_id                    description  \
0         146889328                            AML   
1         131552308                            AML   
2         434354261                Prostate Cancer   
3         130746882                            AML   
4          78415005                   Liver Cancer   
5          56972591                Prostate Cancer   
6         263619129                            AML   
7         166312897                            AML   
8         289176034                  Breast Cancer   
9         313346685                            AML   
10        281830565                            AML   
11        103195342                  Breast Cancer   
12        322000857                            AML   
13        262188474                            AML   
14        160061191              Colorectal Cancer   
15        346170679                            AML   
16        279659460                            AML   
17        230833101               Multiple Myeloma   
18        388268616                    Lung Cancer   
19        406250933                            AML   
20        149650607               Multiple Myeloma   
21        300336593                            AML   
22         47771830               Multiple Myeloma   
23        320132060                            AML   
24        375420401                            AML   
25        270815402              Pancreatic Cancer   
26        281030247               Multiple Myeloma   
27        278851807                            AML   
28        100515286                            AML   
29        374131869                            AML   
...             ...                            ...   
3692036   301105415              Colorectal Cancer   
3692037   192223846                  Breast Cancer   
3692038   309525182                    Lung Cancer   
3692039   194051578                Prostate Cancer   
3692040   416453074                    Lung Cancer   
3692041    87060938                Prostate Cancer   
3692042   242423432                  Breast Cancer   
3692043   505593350  Other Digestive Tract Cancers   
3692044   138473813                 Stomach Cancer   
3692045   248621867              Colorectal Cancer   
3692046   417438145                 Stomach Cancer   
3692047   225941944                  Breast Cancer   
3692048   117527374                Prostate Cancer   
3692049   270725703                Prostate Cancer   
3692050   430571237                  Breast Cancer   
3692051   193578039                Prostate Cancer   
3692052   152380876                  Breast Cancer   
3692053   225776246                Prostate Cancer   
3692054   478655368                  Breast Cancer   
3692055   372093117                Prostate Cancer   
3692056   368665363                  Breast Cancer   
3692057    12735381              Colorectal Cancer   
3692058    35093789                Prostate Cancer   
3692059   201215038              Pancreatic Cancer   
3692060    34411223                    Lung Cancer   
3692061   281361516                Prostate Cancer   
3692062   262678600                Prostate Cancer   
3692063    88597075              Colorectal Cancer   
3692064   193813563                Prostate Cancer   
3692065   289140835                  Breast Cancer   

         COUNT(DISTINCT CLAIM_NUMBER)  
0                                1007  
1                                 780  
2                                 661  
3                                 656  
4                                 614  
5                                 598  
6                                 595  
7                                 588  
8                                 582  
9                                 581  
10                                574  
11                                555  
12                                546  
13                                537  
14                                534  
15                  

## Pulling data for aggregate speciality

This table contains all physician from healthbase

In [142]:
%%read_sql
Create or replace table  Aggr_speciality_1
            as
           Select PHYSICIAN_NPI,
           Case when Upper(AGG_SPECIALTY) like '%ONCO%' then 'ONCOLOGY'
                when Upper(AGG_SPECIALTY) like '%HEMA%' then 'HEMATOLOGY'
                When Upper(AGG_SPECIALTY) like '%INTERNAL%' then 'INTERNAL MEDICINE'
               when Upper(AGG_SPECIALTY) like '%SUR%' then 'SURGERY'
                else PRIMARY_SPECIALITY
                END AS SPECIALTY
                from Aggr_speciality

Query started at 08:58:58 PM India Standard Time; Query executed in 0.06 m

status
0  Table AGGR_SPECIALITY_1 successfully created.

## Assigning speciality to each physician 

In [143]:
%%read_sql
Create or replace table Amgen_TARGET_NPI_SpecialityFlag
AS
Select 
A.*,
B.SPECIALTY as Speciality_flag
From Amgen_provider A Left join Aggr_speciality_1 B 
ON  A.physician_npi=B.physician_npi

Query started at 08:59:01 PM India Standard Time; Query executed in 0.33 m

status
0  Table AMGEN_TARGET_NPI_SPECIALITYFLAG successf...

In [144]:
%%read_sql

select count(distinct claim_number),count(distinct patient_id),count(distinct physician_npi) 
from Amgen_TARGET_NPI_SpecialityFlag 
where physician_npi is not null and Speciality_Flag is not null


Query started at 08:59:21 PM India Standard Time; Query executed in 0.13 m

COUNT(DISTINCT CLAIM_NUMBER)  COUNT(DISTINCT PATIENT_ID)  \
0                      28024586                     3506517   

   COUNT(DISTINCT PHYSICIAN_NPI)  
0                         681824

### Assigning one claim to each physician based on speciality and provider type

In [145]:
%%read_sql

create or replace table Amgen_TARGET_One_physican_1 
As                                                                          
 select *,
 case when Upper(Speciality_Flag) in ('ONCOLOGY') then 1
 when Upper(Speciality_Flag) in ('HEMATOLOGY') then 2
 when  Upper(Speciality_Flag) in ('INTERNAL MEDICINE')   then 3
 when Upper(Speciality_Flag) in ('SURGERY')  then 4

  when  Upper(Speciality_Flag) not in ('INTERNAL MEDICINE','ONCOLOGY','HEMATOLOGY','SURGERY')   then 5
  else 9 end as specialty_rnk
 , case when lower(provider_type)='rendering' then 1
  when lower(provider_type)='attending' then 2
 when lower(provider_type)='operating' then 3
 when lower(provider_type)='referring' then 4
  else 9 end as provider_type_rnk,
 row_number() over (partition by patient_id, claim_number order by specialty_rnk,provider_type_rnk) as rnk                                                                                
 from Amgen_TARGET_NPI_SpecialityFlag
 



Query started at 08:59:29 PM India Standard Time; Query executed in 0.70 m

status
0  Table AMGEN_TARGET_ONE_PHYSICAN_1 successfully...

In [146]:
%%read_sql

create or replace table Amgen_TARGET_One_physican_2 
As
 select *
 from Amgen_TARGET_One_physican_1 
 where rnk=1

Query started at 09:00:11 PM India Standard Time; Query executed in 0.25 m

status
0  Table AMGEN_TARGET_ONE_PHYSICAN_2 successfully...

### Summary of patient and claim number from Amgen_TARGET_One_physican_2 table

In [147]:
%%read_sql
select SPECIALITY_FLAG,provider_type,count(distinct patient_id)
 from Amgen_TARGET_One_physican_2
 where SPECIALITY_FLAG='ONCOLOGY'
 group by 1,2
 order by SPECIALITY_FLAG

Query started at 09:00:26 PM India Standard Time; Query executed in 0.07 m

speciality_flag    provider_type  COUNT(DISTINCT PATIENT_ID)
0         ONCOLOGY  ServiceLocation                          53
1         ONCOLOGY        Referring                      685578
2         ONCOLOGY        Operating                        4544
3         ONCOLOGY          Billing                       13423
4         ONCOLOGY      Supervising                         626
5         ONCOLOGY        Attending                      431277
6         ONCOLOGY            Other                        2725
7         ONCOLOGY         Facility                        1889
8         ONCOLOGY        Rendering                     1064854
9         ONCOLOGY   OtherOperating                         251
10        ONCOLOGY      PrimaryCare                           5

In [148]:
%%read_sql 
 select count(*),count(distinct claim_number ),count(distinct patient_id ),count(distinct Physician_NPI )
 
 from Amgen_TARGET_One_physican_2
 

Query started at 09:00:30 PM India Standard Time; Query executed in 0.11 m

COUNT(*)  COUNT(DISTINCT CLAIM_NUMBER )  COUNT(DISTINCT PATIENT_ID )  \
0  28055150                       28055150                      3509410   

   COUNT(DISTINCT PHYSICIAN_NPI )  
0                          573927

In [149]:
%%read_sql
select * from Amgen_TARGET_One_physican_2 where SPECIALITY_FLAG is null

Query started at 09:00:37 PM India Standard Time; Query executed in 0.30 m

claim_number  patient_id diagnosis  \
0                            288988000000006720   266740782     C3480   
1                         con_EP111418783996116   255853003       C61   
2                         con_EI060518282810764   151080765     C9000   
3                         con_EP053018762875247   377517187    C50812   
4                            290661000000008916   147566400     C3412   
5                         con_EP122117781568313   370733917       C61   
6                         con_EP061918733006675   203153426      C170   
7                                 vul_149732019    28819936     C3491   
8                                 vul_183870282   277726176    C50919   
9                         con_EP071318721945595   130962530      C342   
10                        con_EP020918754398393   249264957       C61   
11                        con_EI110918268676935    91275079     C3490   
12                        con_EI060618289415818   244963617       C61   
13                        con_EI060818296903823   320132060     C9200   
14                        con_EI072718272022223   102173748       C61   
15                        con_EP082518776534790   332945889     C3411   
16     vul_48799b2a-c447-4b91-b4f9-f6477b029e60    24206812      C220   
17                        con_EI042318223922618   362950567     C3431   
18                        con_EP040918771315737    31179814    C50912   
19                        con_EP020918756103140   197603242       C20   
20                           293155000000325715    38192539    C50412   
21                                vul_143482745   285591193     C3491   
22     vul_2c6892d9-3937-42d5-b421-c78ce94bf30c   359764850    C50912   
23                        con_EP111717763807316   154458433       C61   
24                        con_EI112717292007432   191672107       C61   
25                        con_EI040618266228581   364291754    C50919   
26     vul_23b301ca-04ee-4597-bd53-fb4420a210c5   193428715    C50912   
27                           298229000000238429   163759894       C20   
28                        con_EP031218768648068   127295056    C50511   
29                        con_EP010918737501572   272336567     C3432   
...                                         ...         ...       ...   
30534  vul_32294c06-6935-4d7e-beb4-906ab81c7f0d   260963137     C3491   
30535                        287055000000118654   156622940     C3492   
30536                        297957000000517248    71186674    C50011   
30537                     con_EI021518278110050   396755519       C61   
30538                     con_EP100118706233740   282212290     C9000   
30539                     con_EP101218757911929    50340107       C61   
30540  vul_088412c0-4387-4272-b0a9-86185cd20484   266953980       C61   
30541                        286350000000192226   399532188     C3480   
30542                     con_EP090618716730626   262262471      C185   
30543                     con_EP042318723850428   218938106    C50911   
30544                     con_EP102718714127670   241298314    C50311   
30545                     con_EP102418702216692   303259819     C3490   
30546  vul_1494da27-0d9b-4042-99eb-c237be29ca4b   255016576    C50919   
30547                     con_EI121018273049123   176338405       C61   
30548  vul_175aed73-1efe-4717-96c5-c08ad932d4ea   116595889    C50912   
30549                        290699000000072410   176997286     C3491   
30550                     con_EI062018238292592    49791152     C9002   
30551                        289773000000012122    55570978     C3411   
30552  vul_1a0480b2-b23d-4385-aef8-1958e5da3942   133202313      C189   
30553                     con_EI121918211129304   293087663     C3492   
30554                        292131000000436908   213063888    C50912   
30555  vul_6c657e83-c3c0-4fec-a515-ae918102a25d    62311949       C61   
30556                     con_EP060118770753758   161715717       C61   
30557               

In [150]:
%%read_sql
Select count(distinct PHYSICIAN_NPI) from Amgen_TARGET_One_physican_2

Query started at 09:00:55 PM India Standard Time; Query executed in 0.09 m

COUNT(DISTINCT PHYSICIAN_NPI)
0                         573927

In [151]:
%%read_sql
select count(*),count(claim_number),count( distinct PHYSICIAN_NPI)
from Amgen_TARGET_One_physican_2 

Query started at 09:01:01 PM India Standard Time; Query executed in 0.06 m

COUNT(*)  COUNT(CLAIM_NUMBER)  COUNT( DISTINCT PHYSICIAN_NPI)
0  28055150             28055150                          573927

### Counts by speciality

In [152]:
%%read_sql
select SPECIALITY_FLAG,count(distinct claim_number),count(distinct Patient_id)
from Amgen_TARGET_One_physican_2 
group by SPECIALITY_FLAG
order by count(distinct claim_number) desc

Query started at 09:01:05 PM India Standard Time; Query executed in 0.11 m

speciality_flag  \
0                                             ONCOLOGY   
1                                    INTERNAL MEDICINE   
2                                              SURGERY   
3                                              UROLOGY   
4                                      FAMILY MEDICINE   
5                                           SPECIALIST   
6                                 DIAGNOSTIC RADIOLOGY   
7                                               FAMILY   
8                                   NURSE PRACTITIONER   
9              ANATOMIC PATHOLOGY & CLINICAL PATHOLOGY   
10                                 PHYSICIAN ASSISTANT   
11                                  EMERGENCY MEDICINE   
12                                   PULMONARY DISEASE   
13   STUDENT IN AN ORGANIZED HEALTH CARE EDUCATION/...   
14                                        ADULT HEALTH   
15                                          HEMATOLOGY   
16                                    GASTROENTEROLOGY   
17                              CARDIOVASCULAR DISEASE   
18                                         HOSPITALIST   
19                                  GERIATRIC MEDICINE   
20                                             MEDICAL   
21                               THERAPEUTIC RADIOLOGY   
22                                      ANESTHESIOLOGY   
23                                          NEPHROLOGY   
24                 VASCULAR & INTERVENTIONAL RADIOLOGY   
25                                  ANATOMIC PATHOLOGY   
26                                  INFECTIOUS DISEASE   
27             NURSE ANESTHETIST, CERTIFIED REGISTERED   
28                                    GENERAL PRACTICE   
29                     HOSPICE AND PALLIATIVE MEDICINE   
..                                                 ...   
353                           REHABILITATION COUNSELOR   
354                               PSYCHIATRIC HOSPITAL   
355                                              DOULA   
356                                    MUSIC THERAPIST   
357                                          NAPRAPATH   
358                                     ADULT DAY CARE   
359                                          ORTHOTIST   
360                                      PERIOPERATIVE   
361                                               PAIN   
362  COMMUNITY BASED RESIDENTIAL TREATMENT, MENTAL ...   
363            PSYCH/MENTAL HEALTH, CHILD & ADOLESCENT   
364                                  INFECTION CONTROL   
365                          COMMUNITY/RETAIL PHARMACY   
366                  ASSISTIVE TECHNOLOGY PRACTITIONER   
367                                          HOMEOPATH   
368                      HEARING INSTRUMENT SPECIALIST   
369                              SLEEP SPECIALIST, PHD   
370                         ASSISTANT BEHAVIOR ANALYST   
371                                   PHYSICAL THERAPY   
372                                  NUTRITION SUPPORT   
373                                          GERIATRIC   
374                          GENETICIST, MEDICAL (PHD)   
375                                   MATERNAL NEWBORN   
376               DAY TRAINING/HABILITATION SPECIALIST   
377                     TECHNICIAN, HEALTH INFORMATION   
378                                          AMBULANCE   
379                               DENTAL PUBLIC HEALTH   
380                             PEDIATRIC CHIROPRACTOR   
381                                       ART, MEDICAL   
382                                     LONG-TERM CARE   

     COUNT(DISTINCT CLAIM_NUMBER)  COUNT(DISTINCT PATIENT_ID)  
0                        14431817                     1592483  
1                         3833190                     1074249  
2                         1884709                      556849  
3                         1462070                      492673  
4                         1445413                      534361  
5                          566386                      149

## Pharmacy Data pull based on ndc

In [153]:
%%read_sql
Create OR REPLACE TABLE Amgen_Pharmacy 
AS
Select
'RAVEN_PHARM' AS Datasource,
       Dx.CLAIM_NUMBER,
       Dx.PATIENT_ID,
       Dx.PRODUCT_OR_SERVICE_ID as NDC,
      (Dx.date_of_service) as DIAGNOSIS_DATE,
       dx.prescriber_id as provider_NPI,
       Ref_Dx.category_1,
       Ref_Dx.code_group,
       Ref_Dx.Description
FROM RWD_DB.RWD.RAVEN_PHARMACY AS Dx
   INNER JOIN Amgen_diag_code AS Ref_Dx 
   ON Upper(trim(Dx.PRODUCT_OR_SERVICE_ID))=Upper(trim(Ref_Dx.Code))
   where upper(category_1)='NDC' 
         and length(Dx.PRESCRIBER_ID) = 10
         and Dx.PRESCRIBER_ID not ilike '%A%'
         and Dx.PRESCRIBER_ID not ilike '%B%'
         and Dx.PRESCRIBER_ID not ilike '%L%'
         and Dx.PRESCRIBER_ID not ilike '%*%'
         and Dx.PRESCRIBER_ID not ilike '%+%'
         and Dx.PRESCRIBER_ID not ilike '% %'
         and Dx.PRESCRIBER_ID not ilike '%Y%'
         and Dx.PRESCRIBER_ID not ilike '%X%'
       AND Dx.date_of_service>= '2017-11-01' AND Dx.date_of_service<= '2018-10-31' 

Query started at 09:01:11 PM India Standard Time; Query executed in 0.95 m

status
0  Table AMGEN_PHARMACY successfully created.

### Data pull from procedure table based on ndc

In [154]:
%%read_sql
Create OR REPLACE TABLE Amgen_Ndc_procedure
AS
Select
'RAVEN_MEDICAL' as datasource,
Px.claim_number,
Px.PATIENT_ID ,
Px.NDC,
Px.YEAR_OF_SERVICE as Procedure_date,
 Ref_Px.category_1,
 Ref_Px.code_group,
  Ref_Px.Description
FROM RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROCEDURE AS Px
   INNER JOIN Amgen_diag_code AS Ref_Px 
   ON Upper(trim(Px.NDC))=Upper(trim(Ref_Px.Code))
   where upper(Ref_Px.category_1)='NDC'
   
       AND Px.YEAR_OF_SERVICE>= '2017-11-01' AND Px.YEAR_OF_SERVICE<= '2018-10-31' 


Query started at 09:02:08 PM India Standard Time

Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\zkhan\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\engine\base.py", line 1193, in _execute_context
    context)
  File "C:\Users\zkhan\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 507, in do_execute
    cursor.execute(statement, parameters)
  File "C:\Users\zkhan\AppData\Local\Continuum\anaconda3\lib\site-packages\snowflake\connector\cursor.py", line 542, in execute
    errvalue)
  File "C:\Users\zkhan\AppData\Local\Continuum\anaconda3\lib\site-packages\snowflake\connector\errors.py", line 97, in errorhandler_wrapper
    cursor.errorhandler(connection, cursor, errorclass, errorvalue)
  File "C:\Users\zkhan\AppData\Local\Continuum\anaconda3\lib\site-packages\snowflake\connector\errors.py", line 73, in default_errorhandler
    done_format_msg=errorvalue.get(u'done_format_msg'))
snowflake.connector.errors.ProgrammingError: 390420: IP 119.42.59.

ProgrammingError: (snowflake.connector.errors.ProgrammingError) 390420: IP 119.42.59.67 is not allowed to access Snowflake.  Contact your local security administrator. (Background on this error at: http://sqlalche.me/e/f405)

 ### Fetching provider npi for Amgen_Ndc_procedure table

In [ ]:
%%read_sql
 Create or replace table Amgen_Ndc_procedure_1
        AS
        select A.*,
        B.provider_npi as physician_npi,
        B.provider_type
     from  Amgen_Ndc_procedure A left join RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROVIDER B
     on A.claim_number=B.claim_number and entity_type_code='1'
     where 
          length(B.provider_npi) = 10
         and B.provider_npi not ilike '%A%'
         and B.provider_npi not ilike '%B%'
         and B.provider_npi not ilike '%L%'
         and B.provider_npi not ilike '%*%'
         and B.provider_npi not ilike '%+%'
         and B.provider_npi not ilike '% %'
         and B.provider_npi not ilike '%Y%'
         and B.provider_npi not ilike '%X%'

In [ ]:
%%read_sql
Select count(distinct claim_number),count(distinct patient_id),count(distinct physician_npi)
from Amgen_Ndc_procedure_1

### union of both table containing data based on ndc

In [ ]:
%%read_sql
Create or replace table Amgen_All_Ndc
     AS
    Select 
        Datasource,
       CLAIM_NUMBER,
       PATIENT_ID,
        NDC,
       DIAGNOSIS_DATE,
       category_1,
       code_group,
       Description,
        provider_NPI,
        '' as PROVIDER_TYPE
     from Amgen_Pharmacy
     UNION
     Select 
     datasource,
     claim_number,
     PATIENT_ID ,
     NDC,
     Procedure_date,
     category_1,
     code_group,
      Description,
      PHYSICIAN_NPI,
      PROVIDER_TYPE
      from Amgen_Ndc_procedure_1

In [ ]:
%%read_sql
 select count(distinct patient_id),count(distinct claim_number),count(distinct provider_NPI)
     from Amgen_Pharmacy

In [ ]:
%%read_sql

 select count(distinct patient_id),count(distinct claim_number) ,count(distinct provider_NPI)
     from Amgen_All_Ndc
     

### Data pull from procedure based on cpt codes of interest

In [ ]:
%%read_sql
 CREATE OR REPLACE TABLE Amgen_Procedure AS
SELECT 
'RAVEN_MEDICAL' as datasource,
Px.claim_number,
PATIENT_ID ,
Px.PROCEDURE,
Px.YEAR_OF_SERVICE as Procedure_date,
 Ref_Px.category_1,
 Ref_Px.code_group,
  Ref_Px.Description
FROM RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROCEDURE AS Px
INNER JOIN Amgen_diag_code Ref_Px ON Upper(Trim(Px.Procedure))=Upper(Trim(Ref_Px.Code))
where  upper(category_1)='CPT' 
  AND Px.YEAR_OF_SERVICE>= '2017-11-01' AND Px.YEAR_OF_SERVICE<= '2018-10-31' 

### Fetching provider data 

In [ ]:
 %%read_sql
  Create or replace table Amgen_Target_Procedure_provider
AS
select A.*,
      B.provider_npi as physician_npi,
      B.provider_type
            
from Amgen_Procedure A left join RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROVIDER B
     on A.claim_number=B.claim_number and entity_type_code='1'
    where 
    length(B.provider_npi) = 10
         and B.provider_npi not ilike '%A%'
         and B.provider_npi not ilike '%B%'
         and B.provider_npi not ilike '%L%'
         and B.provider_npi not ilike '%*%'
         and B.provider_npi not ilike '%+%'
         and B.provider_npi not ilike '% %'
         and B.provider_npi not ilike '%Y%'
         and B.provider_npi not ilike '%X%'
    

In [ ]:
%%read_sql

 select count(distinct patient_id),count(distinct claim_number) ,count(distinct physician_npi)
     from AMGEN_TARGET_PROCEDURE_PROVIDER 

### Union of procedure and pharmacy table

In [ ]:
%%read_sql
 Create or replace table Amgen_Ndc_procedure_Master_table
 AS
 Select * from Amgen_Target_Procedure_provider
 union 
 select * from Amgen_All_Ndc

In [ ]:
%%read_sql

 select count(distinct patient_id),count(distinct claim_number) ,count(distinct physician_npi)
     from Amgen_Ndc_procedure_Master_table 

## Implementing logic to assign one patient to one cancer based on max claim counts

In [ ]:
%%read_sql
    
create or replace table Amgen_clm_cnt
 as
 select distinct patient_id,description,count(distinct claim_number) as clm_cnt
 from AMGEN_PROVIDER
 group by patient_id,description
 order  by patient_id desc,3 desc

In [ ]:
%%read_sql
select  count (distinct patient_id)
from amgen_clm_cnt

In [ ]:
%%read_sql
create or replace table Amgen_clm_cnt_1
as
 select patient_id,description,clm_cnt,Max(clm_cnt) as Max_clm_counts,
      row_number() over (partition by patient_id order by clm_cnt desc) as Patient_rank
      from Amgen_clm_cnt
       group by patient_id,description,clm_cnt
      

In [ ]:
%%read_sql
create or replace table Amgen_clm_cnt_2 
as
select * from Amgen_clm_cnt_1 where Patient_rank=1

In [ ]:
%%read_sql
 select count(*),count(distinct patient_id) from Amgen_clm_cnt_2 

### Assigning patient to one cancer with max claim count as ' Y' otherwise 'N'

In [179]:
%%read_sql
 create or replace table Amgen_patient_cancer_Mapping
      as
      select distinct patient_id,
      case  when (description ='Breast Cancer' and Max_clm_counts=clm_cnt) then '1' else '0' end as Breast_Cancer,
      case  when (description ='AML' and Max_clm_counts=clm_cnt) then '1' else '0' end as AML,
      case  when (description ='Prostate Cancer' and Max_clm_counts=clm_cnt) then '1' else '0' end as Prostate_Cancer,
      case  when (description ='Colorectal Cancer' and Max_clm_counts=clm_cnt) then '1' else '0' end as Colorectal_Cancer,
      case  when (description ='Multiple Myeloma' and Max_clm_counts=clm_cnt) then '1' else '0' end as Multiple_Myeloma,
      case  when (description ='Pancreatic Cancer' and Max_clm_counts=clm_cnt) then '1' else '0' end as Pancreatic_Cancer,
      case  when (description ='Small Intestine Cancer' and Max_clm_counts=clm_cnt) then '1' else '0' end as Small_Intestine_Cancer,
      case  when (description ='Other Digestive Tract Cancers' and Max_clm_counts=clm_cnt) then '1' else '0' end as Other_Digestive_Tract_Cancers,
      case  when (description ='Liver Cancer' and Max_clm_counts=clm_cnt) then '1' else '0' end as Liver_Cancer,
      case  when (description ='Lung Cancer' and Max_clm_counts=clm_cnt) then '1' else '0' end as Lung_Cancer,
      case  when (description ='Stomach Cancer' and Max_clm_counts=clm_cnt) then '1' else '0' end as Stomach_Cancer,
      case  when (description ='Esophageal Cancer' and Max_clm_counts=clm_cnt) then '1' else '0' end as Esophageal_Cancer
       from Amgen_clm_cnt_2

Query started at 02:18:10 PM India Standard TimeInitiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
; Query executed in 0.37 m

status
0  Table AMGEN_PATIENT_CANCER_MAPPING successfull...

In [180]:
%%read_sql
 select count(*),count(distinct patient_id) from AMGEN_PATIENT_CANCER_MAPPING  

Query started at 02:18:32 PM India Standard Time; Query executed in 0.05 m

COUNT(*)  COUNT(DISTINCT PATIENT_ID)
0   3509153                     3509153

### Extracting data for overlapped patients from diagnosis and medical 

In [181]:
%%read_sql
create or replace table Amgen_diag_proc_overlap
  AS
  Select 
  A.claim_number,
  A.Patient_id,
  A.diagnosis,
  B.procedure,
  A.description
  from Amgen_provider A inner join Amgen_Ndc_procedure_Master_table B
  on A.Patient_id=b.Patient_id

Query started at 02:18:35 PM India Standard Time; Query executed in 0.73 m

status
0  Table AMGEN_DIAG_PROC_OVERLAP successfully cre...

In [228]:
%%read_sql
Select count(distinct claim_number),count(distinct patient_id)
from Amgen_diag_proc_overlap

Query started at 06:05:53 PM India Standard Time; Query executed in 0.05 m

COUNT(DISTINCT CLAIM_NUMBER)  COUNT(DISTINCT PATIENT_ID)
0                       9703809                      323914

### Adding a flag for overlaped patients in Amgen_patient_cancer_Mapping table

In [183]:
%%read_sql
 Create or replace table Amgen_patient_cancer_Mapping_overlap 
 As
   select distinct A.*,
          case when B.patient_id is not null then '1' else '0' end as Overlaped_patient
          
          from Amgen_patient_cancer_Mapping A left join (select distinct patient_id from Amgen_diag_proc_overlap) B
          on A.patient_id=B.patient_id

Query started at 02:19:25 PM India Standard Time; Query executed in 0.11 m

status
0  Table AMGEN_PATIENT_CANCER_MAPPING_OVERLAP suc...

In [217]:
%%read_sql
 select count(*),count(distinct patient_id),sum(Overlaped_patient) from AMGEN_PATIENT_CANCER_MAPPING_OVERLAP  

Query started at 05:41:31 PM India Standard Time; Query executed in 0.06 m

COUNT(*)  COUNT(DISTINCT PATIENT_ID)  SUM(OVERLAPED_PATIENT)
0   3509153                     3509153                323906.0

## Fetching minimum diagnosis date 

In [185]:
%%read_sql
create or replace table Amgen_patient_decision_marker_1 as
           select A.patient_id,
                 A.DESCRIPTION,
                 First_Diagnosis_date
                  from Amgen_clm_cnt_2 A inner join (select  distinct patient_id,Description,min(diagnosis_date) as First_Diagnosis_date 
                                                            from Amgen_Target_Diagnosis_All_Year
                                                            group by patient_id,Description) B
                
                ON A.patient_id=B.patient_id and A.DESCRIPTION=B.DESCRIPTION

Query started at 02:19:35 PM India Standard Time; Query executed in 0.12 m

status
0  Table AMGEN_PATIENT_DECISION_MARKER_1 successf...

In [218]:
%%read_sql
select count (distinct patient_id) from AMGEN_PATIENT_DECISION_MARKER_1 

Query started at 05:41:51 PM India Standard Time; Query executed in 0.05 m

COUNT (DISTINCT PATIENT_ID)
0                      3509153

In [187]:
%%read_sql
create or replace table Amgen_patient_decision_marker_2 as
                 Select A.*,
                        physician_npi,
                        Speciality_flag
                  from Amgen_patient_decision_marker_1 A inner join  (select distinct patient_id,physician_npi,Speciality_flag from 
                                                                        Amgen_TARGET_NPI_SpecialityFlag
                                                                    where physician_npi is not null
                                                                     ) B
                  ON A.patient_id=B.patient_id

Query started at 02:19:45 PM India Standard Time; Query executed in 0.11 m

status
0  Table AMGEN_PATIENT_DECISION_MARKER_2 successf...

In [220]:
%%read_sql
select count(*),count (distinct patient_id),count (distinct  physician_npi)
  from Amgen_TARGET_NPI_SpecialityFlag
 

Query started at 05:42:44 PM India Standard Time; Query executed in 0.06 m

COUNT(*)  COUNT (DISTINCT PATIENT_ID)  COUNT (DISTINCT  PHYSICIAN_NPI)
0  50227057                      3509410                           684867

In [221]:
%%read_sql
select count(*),count (distinct patient_id)
from AMGEN_PATIENT_DECISION_MARKER_1

Query started at 05:43:30 PM India Standard Time; Query executed in 0.06 m

COUNT(*)  COUNT (DISTINCT PATIENT_ID)
0   3509153                      3509153

In [224]:
%%read_sql
select count(*),count (distinct patient_id)
from AMGEN_PATIENT_DECISION_MARKER_2

Query started at 05:44:16 PM India Standard Time; Query executed in 0.05 m

COUNT(*)  COUNT (DISTINCT PATIENT_ID)
0  10534574                      3509153

In [191]:
%%read_sql
create or replace transient table amgen_marker as
select patient_id, description as cancer from Amgen_clm_cnt_2
 where patient_id in (select distinct patient_id from Amgen_diag_proc_overlap) --  

Query started at 02:20:03 PM India Standard Time; Query executed in 0.13 m

status
0  Table AMGEN_MARKER successfully created.

In [225]:
%%read_sql
select count (distinct patient_id) from amgen_marker

Query started at 05:45:13 PM India Standard Time; Query executed in 0.05 m

COUNT (DISTINCT PATIENT_ID)
0                       323906

In [226]:
%%read_sql
Create or replace table amgen_pat_hi_phar as       
with t1 as     
(
      select 
             a.claim_number,
             a.PATIENT_ID,
             a.date_of_service as service_date,
             upper(a.PATIENT_ID || a.date_of_service || nvl(a.PRODUCT_OR_SERVICE_ID,'') ||nvl(a.type_of_payment,'') || nvl(a.plan_name,'')) as script_id,
             a.PRESCRIBER_ID as Physician_NPI,
             a.PRODUCT_OR_SERVICE_ID as ndc,
             a.response_code
        FROM RWD_DB.RWD.RAVEN_PHARMACY AS a
       INNER JOIN amgen_marker b
             on a.patient_id = b.patient_id
         and Upper(trim(a.PRODUCT_OR_SERVICE_ID)) in (select Upper(trim(Code)) 
                                                        from amgen_diag_code where upper(category_1)='NDC')
         and upper(a.Transaction_code) not in ('B2')
         and year(a.date_of_service) between 2011 and 2018
         and length(a.PRESCRIBER_ID) = 10
         and a.PRESCRIBER_ID not ilike '%A%'
         and a.PRESCRIBER_ID not ilike '%B%'
         and a.PRESCRIBER_ID not ilike '%L%'
         and a.PRESCRIBER_ID not ilike '%*%'
         and a.PRESCRIBER_ID not ilike '%+%'
         and a.PRESCRIBER_ID not ilike '% %'
         and a.PRESCRIBER_ID not ilike '%Y%'
         and a.PRESCRIBER_ID not ilike '%X%' 
)
select a.*, 
             Ref_Dx.category_1,
             Ref_Dx.code_group,
             Ref_Dx.Description
from t1  a
left join amgen_diag_code Ref_Dx
      ON a.ndc = Upper(trim(Ref_Dx.Code))
         ;

Query started at 05:45:50 PM India Standard Time; Query executed in 10.72 m

status
0  Table AMGEN_PAT_HI_PHAR successfully created.

In [227]:
%%read_sql
CREATE or replace table med_amgen_ndc  as 
with t1 as
(
      select 'Raven Medical'  as data_source,
             a.Claim_number,
             a.patient_id,            
             a.year_of_service as service_date,
             trim(a.ndc)  as code_list
        From RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROCEDURE   a   
       inner join amgen_marker b
             on a.patient_id = b.patient_id
             and trim(upper(a.ndc)) in (select Upper(trim(Code)) from amgen_diag_code where upper(category_1)='NDC')
             and year(a.year_of_service) between 2011 and 2018
     union
     select 'Raven Medical'  as data_source,
             a.Claim_number,
             a.patient_id,            
             a.year_of_service as service_date,
             trim(a.procedure) as procedure
        From RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROCEDURE   a   
       inner join amgen_marker b
             on a.patient_id = b.patient_id
             and upper(trim(a.PROCEDURE)) in (select Upper(trim(Code)) from amgen_diag_code 
                                               where upper(category_1)='CPT')
             and year(a.year_of_service) between 2011 and 2018 
)      
select a.*, 
             Ref_Dx.category_1,
             Ref_Dx.code_group,
             Ref_Dx.Description
  from t1 a 
  left join Amgen_diag_code Ref_Dx
    on trim(upper(a.code_list)) = trim(upper(Ref_Dx.code))

Query started at 05:56:33 PM India Standard Time; Query executed in 9.33 m

status
0  Table MED_AMGEN_NDC successfully created.

In [196]:
%%read_sql
create or replace table med_amgen_ndc_phy as

select a.*,
                     b.provider_type,
                     b.provider_npi
                from med_amgen_ndc  a
                left join RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROVIDER  B
                     on a.claim_number = b.claim_number
                     AND b.entity_type_code = 1
                     and length(b.provider_npi) = 10
                     and b.provider_npi not ilike '%A%'
                     and b.provider_npi not ilike '%B%'
                     and b.provider_npi not ilike '%L%'
                     and b.provider_npi not ilike '%*%'
                     and b.provider_npi not ilike '%+%'
                     and b.provider_npi not ilike '% %'
                     and b.provider_npi not ilike '%Y%'
                     and b.provider_npi not ilike '%X%'  

;

Query started at 02:37:00 PM India Standard Time; Query executed in 7.09 m

status
0  Table MED_AMGEN_NDC_PHY successfully created.

In [229]:
%%read_sql

Create or replace transient table amgen_pat_history as 
    select  claim_number, patient_id, provider_npi as physician_npi, service_date, 
           code_list,CODE_GROUP,DESCRIPTION,CATEGORY_1
      from med_amgen_ndc_phy
      union
    select  SCRIPT_ID, patient_id, physician_npi,  service_date, 
             ndc,CODE_GROUP,DESCRIPTION,CATEGORY_1
      from amgen_pat_hi_phar

Query started at 06:05:56 PM India Standard Time; Query executed in 0.22 m

status
0  Table AMGEN_PAT_HISTORY successfully created.

In [230]:
%%read_sql
select count(distinct patient_id) from amgen_pat_history

Query started at 06:06:09 PM India Standard Time; Query executed in 0.06 m

COUNT(DISTINCT PATIENT_ID)
0                      323906

In [231]:
%%read_sql
select distinct patient_id from amgen_marker
except
select distinct patient_id from amgen_pat_history

Query started at 06:06:13 PM India Standard Time; Query executed in 0.05 m

Empty DataFrame
Columns: [patient_id]
Index: []

### Initiator

In [233]:
%%read_sql----add min(service_date)
create or replace transient table amgen_first_phy as
with t1 as 
(
       select distinct a.patient_id, a.physician_npi,a.service_date, b.treatment_start,
                        row_number() over (partition by a.patient_id
                                     order by a.service_date) as rr
         from AMGEN_PAT_HISTORY a
         join (select distinct patient_id, min(service_date) as treatment_start 
                 from AMGEN_PAT_HISTORY
                group by 1) b
           on a.patient_id = b.patient_id
)
(select patient_id, physician_npi,treatment_start from t1 where rr = 1 )

Query started at 06:06:41 PM India Standard Time; Query executed in 0.10 m

status
0  Table AMGEN_FIRST_PHY successfully created.

In [234]:
%%read_sql
Select count(distinct patient_id)
from AMGEN_PAT_HISTORY 

Query started at 06:06:59 PM India Standard Time; Query executed in 0.05 m

COUNT(DISTINCT PATIENT_ID)
0                      323906

In [203]:
%%read_sql
Select count(distinct patient_id)
from AMGEN_FIRST_PHY 

Query started at 02:49:26 PM India Standard Time; Query executed in 0.05 m

COUNT(DISTINCT PATIENT_ID)
0                      323906

In [235]:
%%read_sql
with t1 as 
(
select distinct patient_id from Amgen_patient_cancer_Mapping_overlap
intersect
select distinct patient_id from AMGEN_FIRST_PHY)
select count(distinct patient_id) from t1 limit 10

Query started at 06:07:15 PM India Standard Time; Query executed in 0.08 m

COUNT(DISTINCT PATIENT_ID)
0                      323906

In [244]:
%%read_sql
Create or replace table amgen_provider_flag as 
select distinct a.*, e.treatment_start as treatment_start_date, 
        case when a.physician_npi = b.physician_npi then 1 else 0 end as npi_in_RX ,
        case when a.physician_npi = c.physician_npi then 1 else 0 end as initiator_flag ,
        case when a.physician_npi = d.physician_npi then 1 else 0 end as current_npi_rx 
  from Amgen_patient_decision_marker_2 a
  left join  amgen_pat_history b
       on a.patient_id = b.patient_id
       and a.physician_npi = b.physician_npi
  left join AMGEN_FIRST_PHY  c
       on a.patient_id = c.patient_id
       and a.physician_npi = c.physician_npi
  left join Amgen_Ndc_procedure_Master_table  d
       on a.patient_id = d.patient_id
       and a.physician_npi = d.physician_npi   
  left join AMGEN_FIRST_PHY e
       on a.patient_id = e.patient_id

Query started at 07:02:51 PM India Standard Time; Query executed in 1.74 m

status
0  Table AMGEN_PROVIDER_FLAG successfully created.

In [245]:
%%read_sql
select count(distinct patient_id) from AMGEN_PROVIDER_FLAG 

Query started at 07:04:35 PM India Standard Time; Query executed in 0.06 m

COUNT(DISTINCT PATIENT_ID)
0                     3509153

In [246]:
%%read_sql
alter table amgen_provider_flag
add overlap_flag varchar();

update amgen_provider_flag a
    set a.overlap_flag = b.Overlaped_patient
    from Amgen_patient_cancer_Mapping_overlap b
      where a.patient_id = b.patient_id

Query started at 07:04:39 PM India Standard Time; Query executed in 0.05 mQuery started at 07:04:42 PM India Standard Time; Query executed in 0.15 m

number of rows updated  number of multi-joined rows updated
0                10534574                                    0

In [247]:
%%read_sql
Select count(distinct patient_id) from amgen_provider_flag where overlap_flag='1'

Query started at 07:04:51 PM India Standard Time; Query executed in 0.06 m

COUNT(DISTINCT PATIENT_ID)
0                      323906

In [209]:
%%read_sql
desc table AMGEN_PROVIDER_FLAG

Query started at 02:50:27 PM India Standard Time; Query executed in 0.05 m

name               type    kind null? default primary key  \
0            PATIENT_ID       NUMBER(38,0)  COLUMN     Y    None           N   
1           DESCRIPTION  VARCHAR(16777216)  COLUMN     Y    None           N   
2  FIRST_DIAGNOSIS_DATE               DATE  COLUMN     Y    None           N   
3         PHYSICIAN_NPI       VARCHAR(100)  COLUMN     Y    None           N   
4       SPECIALITY_FLAG  VARCHAR(16777216)  COLUMN     Y    None           N   
5  TREATMENT_START_DATE               DATE  COLUMN     Y    None           N   
6             NPI_IN_RX        NUMBER(1,0)  COLUMN     Y    None           N   
7        INITIATOR_FLAG        NUMBER(1,0)  COLUMN     Y    None           N   
8        CURRENT_NPI_RX        NUMBER(1,0)  COLUMN     Y    None           N   
9          OVERLAP_FLAG  VARCHAR(16777216)  COLUMN     Y    None           N   

  unique key check expression comment  
0          N  None       None    None  
1          N  None       None    None  
2          N  None       None    None  
3          N  None       None    None  
4          N  None       None    None  
5          N  None       None    None  
6          N  None       None    None  
7          N  None       None    None  
8          N  None       None    None  
9          N  None       None    None

In [ ]:
%%read_sql
select count(distinct patient_id) from Amgen_patient_cancer_Mapping_overlap where Overlaped_patient = '1'

In [ ]:
%%read_sql
select distinct * from amgen_provider_flag 
where patient_id = '98047847'

In [ ]:
%%read_sql
select count(distinct patient_id ) from amgen_provider_flag-- where Overlaped_patient = 'Y'
-- except 
-- select distinct patient_id from amgen_provider_flag where overlap_flag = 'Y'

In [ ]:
%%read_sql
select distinct patient_id from Amgen_patient_cancer_Mapping_overlap where Overlaped_patient = '1'
except 
select distinct patient_id from amgen_provider_flag where overlap_flag = '1'

In [ ]:
%%read_sql
select count(distinct patient_id) from amgen_provider_flag
--where overlap_flag = 'Y'

In [ ]:
%%read_sql
select * from AMGEN_FIRST_PHY where patient_id ='328061209'

In [ ]:
%%read_sql
select * from AMGEN_PROVIDER_FLAG  where patient_id ='328061209' order by 4

In [ ]:
%%read_sql
select count(*), count (distinct patient_id),count (distinct physician_npi),count (distinct patient_id, physician_npi) from AMGEN_PATIENT_DECISION_MARKER_2  

In [ ]:
%%read_sql
select count(*), count (distinct patient_id),count (distinct physician_npi),count (distinct patient_id, physician_npi) from AMGEN_PROVIDER_FLAG  

In [240]:
%%read_sql
delete from AMGEN_PROVIDER_FLAG where Description in ('Esophageal Cancer','Small Intestine Cancer','Stomach Cancer','Liver Cancer','Other Digestive Tract Cancers');


Query started at 06:09:33 PM India Standard Time; Query executed in 0.10 m

number of rows deleted
0                  903308

In [241]:
%%read_sql
Create or replace table Treated_Pat_phy
as select * from AMGEN_PROVIDER_FLAG  where overlap_flag='1' and npi_in_rx=1 and INITIATOR_FLAG=1 and CURRENT_NPI_RX=1;

Query started at 06:09:42 PM India Standard Time; Query executed in 0.07 m

status
0  Table TREATED_PAT_PHY successfully created.

In [242]:
%%read_sql
Create or replace table Treated_Pat_NPI_not_initiator
as select * from AMGEN_PROVIDER_FLAG where  overlap_flag='1' and patient_id not in (select patient_id from Treated_Pat_phy);


Query started at 06:09:54 PM India Standard Time; Query executed in 0.10 m

status
0  Table TREATED_PAT_NPI_NOT_INITIATOR successful...

In [243]:
%%read_sql
Create or replace table Patient_Physician as
with res as (

select  *, row_number() over (partition by patient_id, description order by case when  speciality_flag ilike '%ONCO%' then 1
                                                   when speciality_flag ilike '%HEMA%' then 2
                                                   when speciality_flag ilike '%MED%' then 3 
                                                   else 4 end ) as rnk from Treated_Pat_NPI_not_initiator)
                                                   
select *  from res where rnk=1
union 
select *,null as rnk from Treated_Pat_phy;

Query started at 06:10:14 PM India Standard Time; Query executed in 0.07 m

status
0  Table PATIENT_PHYSICIAN successfully created.

In [ ]:
%%read_sql

create or replace table Patient_physician_Master as
 
with res as (
 
select  *, row_number() over (partition by patient_id, description order by case when  speciality_flag ilike '%ONCO%' then 1
                                                   when speciality_flag ilike '%HEMA%' then 2
                                                   when speciality_flag ilike '%MED%' then 3 
                                                   else 4 end ) as rnk from Diag_Pat_Not_RX)
                                                   
select *  from res where rnk=1
union 
select * from patient_physician;

In [ ]:
%%read_sql
Create or replace table target_Physician as

select * from Patient_physician_Master where physician_npi in (select physician_npi from patient_physician);

In [ ]:
%%read_sql
 create or replace table Amgen_count_1 as
             select PHYSICIAN_NPI,DESCRIPTION,SPECIALITY_FLAG,sum(OVERLAP_FLAG) as treated_patient
             ,count(distinct patient_id) as diagnosed_patient
             from target_Physician
             group by PHYSICIAN_NPI,SPECIALITY_FLAG,DESCRIPTION
             order by count(distinct patient_id) desc

In [ ]:
%%read_sql
Create or replace table Amgen_count_2 as
with res AS (
  SELECT physician_npi
        ,Speciality_flag
        ,OBJECT_AGG(description, treated_patient) AS Treated
        ,OBJECT_AGG(description, diagnosed_patient) AS Diagnosed
    FROM Amgen_count_1
   GROUP BY physician_npi
           ,Speciality_flag
)
 
 
SELECT physician_npi
      ,Speciality_flag
      ,Treated:"Breast Cancer"::INTEGER AS Breast_rx
      ,Diagnosed:"Breast Cancer"::INTEGER AS Breast_diag
      ,Treated:"Lung Cancer"::INTEGER AS lung_rx
      ,Diagnosed:"Lung Cancer"::INTEGER AS Lung_diag
      ,Treated:"Colorectal Cancer"::INTEGER AS Colo_rx
      ,Diagnosed:"Colorectal Cancer"::INTEGER AS colo_diag
      ,Treated:"Multiple Myeloma"::INTEGER AS mm_rx
      ,Diagnosed:"Multiple Myeloma"::INTEGER AS mm_diag
      ,Treated:"Pancreatic Cancer"::INTEGER AS pan_rx
      ,Diagnosed:"Pancreatic Cancer"::INTEGER AS pan_diag
      ,Treated:"Prostate Cancer"::INTEGER AS pro_rx
      ,Diagnosed:"Prostate Cancer"::INTEGER AS pro_diag
      ,Treated:"AML"::INTEGER AS aml_rx
      ,Diagnosed:"AML"::INTEGER AS aml_diag
      
  FROM res
 ORDER BY physician_npi, speciality_flag
 

In [ ]:
%%read_sql
create or replace table Amgen_Final_table_1
 as    
 select distinct
            a.PHYSICIAN_NPI,A.SPECIALITY_FLAG,
            B.PHYSICIAN_FIRST_NAME,B.PHYSICIAN_LAST_NAME,B.PHYSICIAN_GENDER,B.PHYSICIAN_PRACTICE_ADDRESS,B.PHYSICIAN_CITY,
            B.PHYSICIAN_STATE,B.PHYSICIAN_ZIP_CODE,B.PHYSICIAN_PHONE,B.PRIMARY_AFFILIATION_FACILITY_NPI,B.PRIMARY_AFFILIATION,PRIMARY_AFFILIATION_TYPE,
            B.PRIMARY_AFFILIATION_NAME,B.PRIMARY_AFFILIATION_ADDRESS,B.PRIMARY_AFFILIATION_CITY,B.PRIMARY_AFFILIATION_STATE,
            B.PRIMARY_AFFILIATION_ZIP_CODE,B.SECONDARY_AFFILIATION_FACILITY_NPI,B.SECONDARY_AFFILIATION,SECONDARY_AFFILIATION_TYPE,B.SECONDARY_AFFILIATION_NAME,
            B.SECONDARY_AFFILIATION_ADDRESS,B.SECONDARY_AFFILIATION_CITY,B.SECONDARY_AFFILIATION_STATE,B.SECONDARY_AFFILIATION_ZIP_CODE,
            coalesce(A.BREAST_DIAG,0)+coalesce(A.LUNG_DIAG,0)+coalesce(A.COLO_DIAG,0)
           +coalesce(A.MM_DIAG,0)+coalesce(A.PAN_DIAG,0)+coalesce(A.PRO_DIAG,0)+coalesce(A.AML_DIAG,0) as TOTAL_CANCER_PATIENTS,
            A.BREAST_DIAG,A.LUNG_DIAG,A.COLO_DIAG,A.MM_DIAG,A.PAN_DIAG,A.PRO_DIAG,A.AML_DIAG,
            coalesce(A.BREAST_RX,0)+coalesce(A.LUNG_RX,0)+coalesce(A.COLO_RX,0)
           +coalesce(A.MM_RX,0)+coalesce(A.PAN_RX,0)+coalesce(A.PRO_RX,0)+coalesce(A.AML_RX,0) as TOTAL_TREATED_PATIENTS,
            A.BREAST_RX,A.LUNG_RX,A.COLO_RX,A.MM_RX,
            A.PAN_RX,A.PRO_RX,A.AML_RX

--a.PATIENT_COUNT
from Amgen_count_2 A Left join AGGR_SPECIALITY B
on a.PHYSICIAN_NPI=b.PHYSICIAN_NPI

In [ ]:
%%read_sql 

create or replace table Amgen_Final_table_2 as
 select * from AMGEN_FINAL_TABLE_1  where upper(speciality_flag) in ('ONCOLOGY',
'INTERNAL MEDICINE',
'SPECIALIST',
'HEMATOLOGY',
'NURSE PRACTITIONER',
'PHYSICIAN ASSISTANT',
'ADULT HEALTH'
)

### Preparing summary count to put in final client presentation

In [ ]:
%%read_sql
 create or replace table summary_table as
        Select A.*,
        B.region
        from AMGEN_FINAL_TABLE_4 A inner join state_to_region B
        on A.PHYSICIAN_STATE=B.STATE

In [ ]:
%%read_sql
 create or replace table summary_table_1 as
        select distinct region from SUMMARY_TABLE where PRIMARY_AFFILIATION_NAME is not null

In [ ]:
%%read_sql

 with a as
   (
   select PRIMARY_AFFILIATION_NAME,count(PHYSICIAN_NPI) as cnt from summary_table_1   where PRIMARY_AFFILIATION_NAME in (select PRIMARY_AFFILIATION_NAME from summary_table_1 
                                                                         where region='Midwest'
                                                                    order by TOTAL_TREATED_PATIENTS desc limit 10) 
                                                                    and region='Midwest'
                                                                    group by PRIMARY_AFFILIATION_NAME
                                                                    order by count(PHYSICIAN_NPI) desc
   ),
   b as
   (
   select PRIMARY_AFFILIATION_NAME,PHYSICIAN_NPI,PHYSICIAN_FIRST_NAME||' '||PHYSICIAN_last_NAME as physician_name,SPECIALITY_FLAG,PHYSICIAN_STATE,TOTAL_TREATED_PATIENTS
        from summary_table_1
        where region='Midwest'
        group by PRIMARY_AFFILIATION_NAME,PHYSICIAN_NPI,physician_name,SPECIALITY_FLAG,PHYSICIAN_STATE,TOTAL_TREATED_PATIENTS
        order by TOTAL_TREATED_PATIENTS desc limit 10
   )
   select b.PRIMARY_AFFILIATION_NAME,b.PHYSICIAN_NPI,b.physician_name,b.SPECIALITY_FLAG,b.PHYSICIAN_STATE,b.TOTAL_TREATED_PATIENTS,,a.cnt
   from b
   inner join a
   on a.PRIMARY_AFFILIATION_NAME=b.PRIMARY_AFFILIATION_NAME;


In [ ]:
%%read_sql
 with a as
   (
   select PRIMARY_AFFILIATION_NAME,count(PHYSICIAN_NPI) as cnt from summary_table_1   where PRIMARY_AFFILIATION_NAME in (select PRIMARY_AFFILIATION_NAME from summary_table_1 
                                                                         where region='West'
                                                                    order by TOTAL_TREATED_PATIENTS desc limit 10) 
                                                                    and region='West'
                                                                    group by PRIMARY_AFFILIATION_NAME
                                                                    order by count(PHYSICIAN_NPI) desc
   ),
   b as
   (
   select PRIMARY_AFFILIATION_NAME,PHYSICIAN_NPI,PHYSICIAN_FIRST_NAME||' '||PHYSICIAN_last_NAME as physician_name,SPECIALITY_FLAG,PHYSICIAN_STATE,TOTAL_TREATED_PATIENTS
        from summary_table_1
        where region='West'
        group by PRIMARY_AFFILIATION_NAME,PHYSICIAN_NPI,physician_name,SPECIALITY_FLAG,PHYSICIAN_STATE,TOTAL_TREATED_PATIENTS
        order by TOTAL_TREATED_PATIENTS desc limit 10
   )
   select b.PRIMARY_AFFILIATION_NAME,b.PHYSICIAN_NPI,b.physician_name,b.SPECIALITY_FLAG,b.PHYSICIAN_STATE,b.TOTAL_TREATED_PATIENTS,a.cnt
   from b
   inner join a
   on a.PRIMARY_AFFILIATION_NAME=b.PRIMARY_AFFILIATION_NAME

In [ ]:
%%read_sql
with a as
   (
   select PRIMARY_AFFILIATION_NAME,count(PHYSICIAN_NPI) as cnt from summary_table_1   where PRIMARY_AFFILIATION_NAME in (select PRIMARY_AFFILIATION_NAME from summary_table_1 
                                                                         where region='Northeast'
                                                                    order by TOTAL_TREATED_PATIENTS desc limit 10) 
                                                                    and region='Northeast'
                                                                    group by PRIMARY_AFFILIATION_NAME
                                                                    order by count(PHYSICIAN_NPI) desc
   ),
   b as
   (
   select PRIMARY_AFFILIATION_NAME,PHYSICIAN_NPI,PHYSICIAN_FIRST_NAME||' '||PHYSICIAN_last_NAME as physician_name,SPECIALITY_FLAG,PHYSICIAN_STATE,TOTAL_TREATED_PATIENTS
        from summary_table_1
        where region='Northeast'
        group by PRIMARY_AFFILIATION_NAME,PHYSICIAN_NPI,physician_name,SPECIALITY_FLAG,PHYSICIAN_STATE,TOTAL_TREATED_PATIENTS
        order by TOTAL_TREATED_PATIENTS desc limit 10
   )
   select b.PRIMARY_AFFILIATION_NAME,b.PHYSICIAN_NPI,b.physician_name,b.SPECIALITY_FLAG,b.PHYSICIAN_STATE,b.TOTAL_TREATED_PATIENTS,a.cnt
   from b
   inner join a
   on a.PRIMARY_AFFILIATION_NAME=b.PRIMARY_AFFILIATION_NAME

In [ ]:
%%read_sql

        with a as
   (
   select PRIMARY_AFFILIATION_NAME,count(PHYSICIAN_NPI) as cnt from summary_table_1   where PRIMARY_AFFILIATION_NAME in (select PRIMARY_AFFILIATION_NAME from summary_table_1 
                                                                         where region='South'
                                                                    order by TOTAL_TREATED_PATIENTS desc limit 10) 
                                                                    and region='South'
                                                                    group by PRIMARY_AFFILIATION_NAME
                                                                    order by count(PHYSICIAN_NPI) desc
   ),
   b as
   (
   select PRIMARY_AFFILIATION_NAME,PHYSICIAN_NPI,PHYSICIAN_FIRST_NAME||' '||PHYSICIAN_last_NAME as physician_name,SPECIALITY_FLAG,PHYSICIAN_STATE,TOTAL_TREATED_PATIENTS
        from summary_table_1
        where region='South'
        group by PRIMARY_AFFILIATION_NAME,PHYSICIAN_NPI,physician_name,SPECIALITY_FLAG,PHYSICIAN_STATE,TOTAL_TREATED_PATIENTS
        order by TOTAL_TREATED_PATIENTS desc limit 10
   )
   select b.PRIMARY_AFFILIATION_NAME,b.PHYSICIAN_NPI,b.physician_name,b.SPECIALITY_FLAG,b.PHYSICIAN_STATE,b.TOTAL_TREATED_PATIENTS,a.cnt
   from b
   inner join a
   on a.PRIMARY_AFFILIATION_NAME=b.PRIMARY_AFFILIATION_NAME;
   

### Calculating cumulativer sum and decile for all treated patients as well as for all individual cancer types

In [ ]:
%%read_sql
create or replace table Amgen_Final_table_3 as
select *,sum(TOTAL_TREATED_PATIENTS) over (order by TOTAL_TREATED_PATIENTS 
                                             rows between unbounded preceding and current row) as TOTAL_TREATED_PATIENTS_Cumalative_sum
  from Amgen_Final_table_2

In [ ]:
%%read_sql
create or replace table Amgen_Final_table_4 as
select *,
   case when TOTAL_TREATED_PATIENTS_Cumalative_sum<=246403/10 then 1
        when TOTAL_TREATED_PATIENTS_Cumalative_sum>246403/10 and  TOTAL_TREATED_PATIENTS_Cumalative_sum<=246403/5 then 2
        when TOTAL_TREATED_PATIENTS_Cumalative_sum>246403/5 and  TOTAL_TREATED_PATIENTS_Cumalative_sum<=246403*(3/10) then 3
        when TOTAL_TREATED_PATIENTS_Cumalative_sum>246403*(3/10) and  TOTAL_TREATED_PATIENTS_Cumalative_sum<=246403*(4/10) then 4
        when TOTAL_TREATED_PATIENTS_Cumalative_sum>246403*(4/10) and  TOTAL_TREATED_PATIENTS_Cumalative_sum<=246403*(5/10) then 5
        when TOTAL_TREATED_PATIENTS_Cumalative_sum>246403*(5/10) and  TOTAL_TREATED_PATIENTS_Cumalative_sum<=246403*(6/10) then 6
        when TOTAL_TREATED_PATIENTS_Cumalative_sum>246403*(6/10) and  TOTAL_TREATED_PATIENTS_Cumalative_sum<=246403*(7/10) then 7
        when TOTAL_TREATED_PATIENTS_Cumalative_sum>246403*(7/10) and  TOTAL_TREATED_PATIENTS_Cumalative_sum<=246403*(8/10) then 8
        when TOTAL_TREATED_PATIENTS_Cumalative_sum>246403*(8/10) and  TOTAL_TREATED_PATIENTS_Cumalative_sum<=246403*(9/10) then 9
         else 10       
        end as Decile
        from Amgen_Final_table_3

In [ ]:
%%read_sql
create or replace table Amgen_Final_table_3_1 as
select *,sum(BREAST_RX) over (order by BREAST_RX
                                             rows between unbounded preceding and current row) as BREAST_RX_Cumalative_sum
  from Amgen_Final_table_3

In [ ]:
%%read_sql
create or replace table Amgen_Final_table_3_2 as
select *,sum(LUNG_RX) over (order by LUNG_RX
                                             rows between unbounded preceding and current row) as LUNG_RX_Cumalative_sum
  from Amgen_Final_table_3_1

In [ ]:
 %%read_sql
    create or replace table Amgen_Final_table_3_3 as
select *,sum(COLO_RX) over (order by COLO_RX
                                             rows between unbounded preceding and current row) as COLO_RX_Cumalative_sum
  from Amgen_Final_table_3_2

In [ ]:
%%read_sql
create or replace table Amgen_Final_table_3_4 as
select *,sum(MM_RX) over (order by MM_RX
                                             rows between unbounded preceding and current row) as MM_RX_Cumalative_sum
  from Amgen_Final_table_3_3

In [ ]:
%%read_sql
create or replace table Amgen_Final_table_3_5 as
select *,sum(PAN_RX) over (order by PAN_RX
                                             rows between unbounded preceding and current row) as PAN_RX_Cumalative_sum
  from Amgen_Final_table_3_4

In [ ]:
%%read_sql
create or replace table Amgen_Final_table_3_6 as
select *,sum(PRO_RX) over (order by PRO_RX
                                             rows between unbounded preceding and current row) as PRO_RX_Cumalative_sum
  from Amgen_Final_table_3_5


In [ ]:
%%read_sql
create or replace table Amgen_Final_table_3_7 as
select *,sum(AML_RX) over (order by AML_RX
                                             rows between unbounded preceding and current row) as AML_RX_Cumalative_sum
  from Amgen_Final_table_3_6

In [ ]:
%%read_sql

create or replace table Amgen_Final_table_4_1 as
select *,
   case when BREAST_RX_Cumalative_sum<=71894/10 then 1
        when BREAST_RX_Cumalative_sum>71894/10 and  BREAST_RX_Cumalative_sum<=71894/5 then 2
        when BREAST_RX_Cumalative_sum>71894/5 and  BREAST_RX_Cumalative_sum<=71894*(3/10) then 3
        when BREAST_RX_Cumalative_sum>71894*(3/10) and  BREAST_RX_Cumalative_sum<=71894*(4/10) then 4
        when BREAST_RX_Cumalative_sum>71894*(4/10) and  BREAST_RX_Cumalative_sum<=71894*(5/10) then 5
        when BREAST_RX_Cumalative_sum>71894*(5/10) and  BREAST_RX_Cumalative_sum<=71894*(6/10) then 6
        when BREAST_RX_Cumalative_sum>71894*(6/10) and  BREAST_RX_Cumalative_sum<=71894*(7/10) then 7
        when BREAST_RX_Cumalative_sum>71894*(7/10) and  BREAST_RX_Cumalative_sum<=71894*(8/10) then 8
        when BREAST_RX_Cumalative_sum>71894*(8/10) and  BREAST_RX_Cumalative_sum<=71894*(9/10) then 9
         else 10       
        end as Breast_Decile
        from AMGEN_FINAL_TABLE_4 
        

In [ ]:
%%read_sql
create or replace table Amgen_Final_table_4_2 as
select *,
   case when LUNG_RX_Cumalative_sum<=57931/10 then 1
        when LUNG_RX_Cumalative_sum>57931/10 and  LUNG_RX_Cumalative_sum<=57931/5 then 2
        when LUNG_RX_Cumalative_sum>57931/5 and  LUNG_RX_Cumalative_sum<=57931*(3/10) then 3
        when LUNG_RX_Cumalative_sum>57931*(3/10) and  LUNG_RX_Cumalative_sum<=57931*(4/10) then 4
        when LUNG_RX_Cumalative_sum>57931*(4/10) and  LUNG_RX_Cumalative_sum<=57931*(5/10) then 5
        when LUNG_RX_Cumalative_sum>57931*(5/10) and  LUNG_RX_Cumalative_sum<=57931*(6/10) then 6
        when LUNG_RX_Cumalative_sum>57931*(6/10) and  LUNG_RX_Cumalative_sum<=57931*(7/10) then 7
        when LUNG_RX_Cumalative_sum>57931*(7/10) and  LUNG_RX_Cumalative_sum<=57931*(8/10) then 8
        when LUNG_RX_Cumalative_sum>57931*(8/10) and  LUNG_RX_Cumalative_sum<=57931*(9/10) then 9
         else 10       
        end as Lung_Decile
        from AMGEN_FINAL_TABLE_4_1  

In [ ]:
%%read_sql
         create or replace table Amgen_Final_table_4_3 as
select *,
   case when COLO_RX_Cumalative_sum<=42711/10 then 1
        when COLO_RX_Cumalative_sum>42711/10 and  COLO_RX_Cumalative_sum<=42711/5 then 2
        when COLO_RX_Cumalative_sum>42711/5 and  COLO_RX_Cumalative_sum<=42711*(3/10) then 3
        when COLO_RX_Cumalative_sum>42711*(3/10) and  COLO_RX_Cumalative_sum<=42711*(4/10) then 4
        when COLO_RX_Cumalative_sum>42711*(4/10) and  COLO_RX_Cumalative_sum<=42711*(5/10) then 5
        when COLO_RX_Cumalative_sum>42711*(5/10) and  COLO_RX_Cumalative_sum<=42711*(6/10) then 6
        when COLO_RX_Cumalative_sum>42711*(6/10) and  COLO_RX_Cumalative_sum<=42711*(7/10) then 7
        when COLO_RX_Cumalative_sum>42711*(7/10) and  COLO_RX_Cumalative_sum<=42711*(8/10) then 8
        when COLO_RX_Cumalative_sum>42711*(8/10) and  COLO_RX_Cumalative_sum<=42711*(9/10) then 9
         else 10       
        end as COLO_Decile
        from AMGEN_FINAL_TABLE_4_2  

In [ ]:

         create or replace table Amgen_Final_table_4_4 as
select *,
   case when MM_RX_Cumalative_sum<=25151/10 then 1
        when MM_RX_Cumalative_sum>25151/10 and  MM_RX_Cumalative_sum<=25151/5 then 2
        when MM_RX_Cumalative_sum>25151/5 and  MM_RX_Cumalative_sum<=25151*(3/10) then 3
        when MM_RX_Cumalative_sum>25151*(3/10) and  MM_RX_Cumalative_sum<=25151*(4/10) then 4
        when MM_RX_Cumalative_sum>25151*(4/10) and  MM_RX_Cumalative_sum<=25151*(5/10) then 5
        when MM_RX_Cumalative_sum>25151*(5/10) and  MM_RX_Cumalative_sum<=25151*(6/10) then 6
        when MM_RX_Cumalative_sum>25151*(6/10) and  MM_RX_Cumalative_sum<=25151*(7/10) then 7
        when MM_RX_Cumalative_sum>25151*(7/10) and  MM_RX_Cumalative_sum<=25151*(8/10) then 8
        when MM_RX_Cumalative_sum>25151*(8/10) and  MM_RX_Cumalative_sum<=25151*(9/10) then 9
         else 10       
        end as MM_Decile
        from AMGEN_FINAL_TABLE_4_3 

In [ ]:
%%read_sql
create or replace table Amgen_Final_table_4_5 as
select *,
   case when PAN_RX_Cumalative_sum<=15215/10 then 1
        when PAN_RX_Cumalative_sum>15215/10 and  PAN_RX_Cumalative_sum<=15215/5 then 2
        when PAN_RX_Cumalative_sum>15215/5 and  PAN_RX_Cumalative_sum<=15215*(3/10) then 3
        when PAN_RX_Cumalative_sum>15215*(3/10) and  PAN_RX_Cumalative_sum<=15215*(4/10) then 4
        when PAN_RX_Cumalative_sum>15215*(4/10) and  PAN_RX_Cumalative_sum<=15215*(5/10) then 5
        when PAN_RX_Cumalative_sum>15215*(5/10) and  PAN_RX_Cumalative_sum<=15215*(6/10) then 6
        when PAN_RX_Cumalative_sum>15215*(6/10) and  PAN_RX_Cumalative_sum<=15215*(7/10) then 7
        when PAN_RX_Cumalative_sum>15215*(7/10) and  PAN_RX_Cumalative_sum<=15215*(8/10) then 8
        when PAN_RX_Cumalative_sum>15215*(8/10) and  PAN_RX_Cumalative_sum<=15215*(9/10) then 9
         else 10       
        end as PAN_Decile
        from AMGEN_FINAL_TABLE_4_4

In [ ]:
 %%read_sql
    
    create or replace table Amgen_Final_table_4_6 as
select *,
   case when PRO_RX_Cumalative_sum<=27070/10 then 1
        when PRO_RX_Cumalative_sum>27070/10 and  PRO_RX_Cumalative_sum<=27070/5 then 2
        when PRO_RX_Cumalative_sum>27070/5 and  PRO_RX_Cumalative_sum<=27070*(3/10) then 3
        when PRO_RX_Cumalative_sum>27070*(3/10) and  PRO_RX_Cumalative_sum<=27070*(4/10) then 4
        when PRO_RX_Cumalative_sum>27070*(4/10) and  PRO_RX_Cumalative_sum<=27070*(5/10) then 5
        when PRO_RX_Cumalative_sum>27070*(5/10) and  PRO_RX_Cumalative_sum<=27070*(6/10) then 6
        when PRO_RX_Cumalative_sum>27070*(6/10) and  PRO_RX_Cumalative_sum<=27070*(7/10) then 7
        when PRO_RX_Cumalative_sum>27070*(7/10) and  PRO_RX_Cumalative_sum<=27070*(8/10) then 8
        when PRO_RX_Cumalative_sum>27070*(8/10) and  PRO_RX_Cumalative_sum<=27070*(9/10) then 9
         else 10       
        end as PRO_Decile
        from AMGEN_FINAL_TABLE_4_5
        

In [ ]:
%%read_sql
         create or replace table Amgen_Final_table_4_7 as
select *,
   case when AML_RX_Cumalative_sum<=6431/10 then 1
        when AML_RX_Cumalative_sum>6431/10 and  AML_RX_Cumalative_sum<=6431/5 then 2
        when AML_RX_Cumalative_sum>6431/5 and  AML_RX_Cumalative_sum<=6431*(3/10) then 3
        when AML_RX_Cumalative_sum>6431*(3/10) and  AML_RX_Cumalative_sum<=6431*(4/10) then 4
        when AML_RX_Cumalative_sum>6431*(4/10) and  AML_RX_Cumalative_sum<=6431*(5/10) then 5
        when AML_RX_Cumalative_sum>6431*(5/10) and  AML_RX_Cumalative_sum<=6431*(6/10) then 6
        when AML_RX_Cumalative_sum>6431*(6/10) and  AML_RX_Cumalative_sum<=6431*(7/10) then 7
        when AML_RX_Cumalative_sum>6431*(7/10) and  AML_RX_Cumalative_sum<=6431*(8/10) then 8
        when AML_RX_Cumalative_sum>6431*(8/10) and  AML_RX_Cumalative_sum<=6431*(9/10) then 9
         else 10       
        end as AML_Decile
        from AMGEN_FINAL_TABLE_4_6
        